# Quantum Error Correction - Steane Code for Fault-Tolerant Quantum Computing

Quantum Error Correction (QEC) is essential for building reliable quantum computers. This notebook explores the Steane [[7,1,3]] code, a fundamental quantum error correction code that can detect and correct arbitrary single-qubit errors.

In [ ]:
# Import required libraries
import time

from ariadne import explain_routing, simulate
from ariadne.algorithms import AlgorithmParameters, SteaneCode

## Understanding the Steane Code

The Steane code is a [[7,1,3]] CSS (Calderbank-Shor-Steane) quantum error correction code:

- **[[n,k,d]] = [[7,1,3]]**: 7 physical qubits encode 1 logical qubit with distance 3
- **Distance 3**: Can detect and correct any single-qubit error
- **CSS Structure**: Separate X and Z stabilizer measurements

Key components:
1. **Encoding**: Maps logical |0⟩_L and |1⟩_L to 7-qubit states
2. **Stabilizers**: 6 operators that detect errors without disturbing logical information
3. **Syndrome measurement**: Extracts error information
4. **Recovery**: Applies correction based on syndrome

## Creating Steane Code Circuits

In [ ]:
# Create Steane code circuits for different scenarios
print("Creating Steane Code circuits:")
print("=" * 40)

# Basic encoding and error detection
params_basic = AlgorithmParameters(n_qubits=7)
steane_basic = SteaneCode(params_basic)
circuit_basic = steane_basic.create_circuit()

print("Basic Steane Code Circuit:")
print(f"  Total qubits: {circuit_basic.num_qubits} (7 data + 6 ancilla)")
print(f"  Depth: {circuit_basic.depth()}")
print(f"  Gate counts: {circuit_basic.count_ops()}")
print()

# With introduced error for demonstration
params_error = AlgorithmParameters(
    n_qubits=7, custom_params={"introduce_error": True, "error_qubit": 0, "error_type": "X"}
)
steane_error = SteaneCode(params_error)
circuit_error = steane_error.create_circuit()

print("Steane Code with X Error on Qubit 0:")
print(f"  Total qubits: {circuit_error.num_qubits}")
print(f"  Depth: {circuit_error.depth()}")
print(f"  Gate counts: {circuit_error.count_ops()}")
print()

# Store circuits for later use
steane_circuits = {"basic": circuit_basic, "with_error": circuit_error}

## Visualizing Steane Code Structure

In [ ]:
# Display a simplified version of the Steane code circuit
print("Simplified Steane Code Circuit Structure:")
print("(Showing first 50 operations due to circuit size)")
print()
print(circuit_basic.draw(output="text", fold=50))

## Testing Steane Code Across Different Backends

In [ ]:
# Test Steane code across different backends
test_circuit = circuit_basic
backends = ["stim", "qiskit", "mps", "tensor_network"]
results = {}

print("Testing Steane Code across backends:")
print("=" * 50)

for backend in backends:
    try:
        start_time = time.time()
        result = simulate(test_circuit, shots=1000, backend=backend)
        end_time = time.time()

        results[backend] = {
            "time": end_time - start_time,
            "counts": result.counts,
            "backend_used": result.backend_used.value,
        }

        print(f"{backend:15} | {end_time - start_time:8.4f}s | {result.backend_used.value}")

    except Exception as e:
        print(f"{backend:15} | FAILED - {str(e)[:30]}")
        results[backend] = {"error": str(e)}

## Analyzing Syndrome Measurements

In [ ]:
# Analyze the syndrome measurement results
print("\nSteane Code Syndrome Analysis:")
print("=" * 40)
print("Qubit layout: [0-6: data qubits, 7-12: X syndrome, 13+: Z syndrome]")
print()

for backend, data in results.items():
    if "error" not in data:
        counts = data["counts"]
        total_shots = sum(counts.values())

        print(f"\n{backend.upper()} Results:")

        # Analyze syndrome patterns
        syndrome_patterns = {}

        for outcome, count in counts.items():
            if len(outcome) >= 13:  # Ensure we have enough bits
                # Extract syndrome bits
                data_bits = outcome[:7]  # First 7 bits are data qubits
                x_syndrome = outcome[7:10]  # Next 3 bits are X syndrome
                z_syndrome = outcome[10:13]  # Next 3 bits are Z syndrome

                syndrome = x_syndrome + z_syndrome
                syndrome_patterns[syndrome] = syndrome_patterns.get(syndrome, 0) + count

        print("  Syndrome patterns found:")
        for syndrome, count in sorted(syndrome_patterns.items(), key=lambda x: x[1], reverse=True):
            percentage = (count / total_shots) * 100
            x_part = syndrome[:3]
            z_part = syndrome[3:]
            print(f"    X:{x_part} Z:{z_part} - {count:4d} shots ({percentage:.1f}%)")

## Error Detection and Correction Demonstration

In [ ]:
# Compare circuits with and without errors
print("Error Detection Comparison:")
print("=" * 40)

error_scenarios = [
    {"type": "no_error", "name": "No Error"},
    {"type": "x_error", "name": "X Error on Qubit 0"},
    {"type": "z_error", "name": "Z Error on Qubit 0"},
    {"type": "y_error", "name": "Y Error on Qubit 0"},
]

scenario_results = {}

for scenario in error_scenarios:
    if scenario["type"] == "no_error":
        params = AlgorithmParameters(n_qubits=7)
    else:
        error_type = scenario["type"].split("_")[0].upper()
        params = AlgorithmParameters(
            n_qubits=7, custom_params={"introduce_error": True, "error_qubit": 0, "error_type": error_type}
        )

    steane = SteaneCode(params)
    circuit = steane.create_circuit()

    try:
        result = simulate(circuit, shots=1000, backend="qiskit")
        counts = result.counts

        # Calculate syndrome statistics
        syndrome_stats = {}
        for outcome, count in counts.items():
            if len(outcome) >= 13:
                x_syndrome = outcome[7:10]
                z_syndrome = outcome[10:13]
                syndrome = x_syndrome + z_syndrome
                syndrome_stats[syndrome] = syndrome_stats.get(syndrome, 0) + count

        # Find most common syndrome
        most_common_syndrome = max(syndrome_stats.items(), key=lambda x: x[1])

        scenario_results[scenario["type"]] = {
            "most_common_syndrome": most_common_syndrome[0],
            "frequency": most_common_syndrome[1],
            "total_shots": sum(counts.values()),
        }

        print(f"\n{scenario['name']}:")
        print(f"  Most common syndrome: X:{most_common_syndrome[0][:3]} Z:{most_common_syndrome[0][3:]}")
        print(f"  Frequency: {most_common_syndrome[1]}/{scenario_results[scenario['type']]['total_shots']} shots")

    except Exception as e:
        print(f"\n{scenario['name']}: FAILED - {str(e)[:30]}")

## Steane Code Properties Analysis

In [ ]:
# Analyze circuit properties of the Steane code
print("\nSteane Code Circuit Properties:")
print("=" * 30)

params = AlgorithmParameters(n_qubits=7)
steane = SteaneCode(params)
analysis = steane.analyze_circuit_properties()

print(f"Total qubits: {analysis['n_qubits']}")
print(f"Depth: {analysis['depth']}")
print(f"Total gates: {analysis['size']}")
print(f"Two-qubit gates: {analysis['two_qubit_gates']}")
print(f"Entanglement heuristic: {analysis['entanglement_heuristic']:.2f}")
print()
print("Gate breakdown:")
for gate_type, count in sorted(analysis["gate_counts"].items(), key=lambda x: x[1], reverse=True):
    print(f"  {gate_type}: {count}")

print()
print("Steane Code Specifications:")
print("  [[n,k,d]] = [[7,1,3]]")
print("  Encodes: 1 logical qubit")
print("  Physical qubits: 7")
print("  Distance: 3 (can correct any single-qubit error)")
print("  Rate: k/n = 1/7 ≈ 0.14")
print("  Ancilla qubits needed: 6 (3 for X, 3 for Z syndromes)")

## Understanding Stabilizer Measurements

In [ ]:
# Explain the stabilizer structure of the Steane code
print("Steane Code Stabilizers:")
print("=" * 30)
print()
print("X-type stabilizers (detect bit-flip errors):")
print("  S1 = X1 X2 X3 X4")
print("  S2 = X1 X2 X5 X6")
print("  S3 = X1 X3 X5 X7")
print()
print("Z-type stabilizers (detect phase-flip errors):")
print("  S4 = Z1 Z2 Z3 Z4")
print("  S5 = Z1 Z2 Z5 Z6")
print("  S6 = Z1 Z3 Z5 Z7")
print()
print("Error detection:")
print("  - Each stabilizer measurement gives 1 bit of syndrome")
print("  - 6 total syndrome bits identify error location and type")
print("  - All zeros syndrome = no error detected")
print("  - Non-zero syndrome = specific error identified")
print()
print("Logical states:")
print("  |0⟩_L = (1/√8) Σ_{a∈C} |a⟩ where C is the [7,4,3] Hamming code")
print("  |1⟩_L = X^⊗7 |0⟩_L")

## Educational Content: Steane Code Mathematical Background

In [ ]:
# Get educational content about the Steane code
params = AlgorithmParameters(n_qubits=7)
steane = SteaneCode(params)
educational_content = steane.get_educational_content()

print("=== Steane Code Mathematical Background ===")
print(educational_content["mathematical_background"])

print("\n=== Implementation Notes ===")
print(educational_content["implementation_notes"])

print("\n=== Applications ===")
print(educational_content["applications"])

## Routing Analysis for Steane Code

In [ ]:
# Analyze how Ariadne routes Steane code circuits
print("Routing Analysis for Steane Code:")
print("=" * 30)

explanation = explain_routing(test_circuit)
print(explanation)

## Key Takeaways

1. **Error Protection**: Steane code can detect and correct any single-qubit error
2. **CSS Structure**: Separate X and Z stabilizer measurements simplify implementation
3. **Overhead**: Requires 7 physical qubits to encode 1 logical qubit (7:1 overhead)
4. **Syndrome Extraction**: 6 syndrome bits identify error location and type
5. **Fault Tolerance**: Foundation for building fault-tolerant quantum computers

Quantum Error Correction is essential for practical quantum computing. The Steane code demonstrates the fundamental principles of how quantum information can be protected from errors, enabling reliable quantum computation despite the fragile nature of quantum states.